In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict, Annotated
from pydantic import BaseModel, Field
from dotenv import load_dotenv 
import os

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


c:\projects\Job-Gap-Analyzer\venv\lib\site-packages\google\api_core\_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.5) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)


In [ ]:
from tavily import TavilyClient
tavily_client = TavilyClient(api_key="")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key="")

In [ ]:
job_title = "Data Scientist"
years_of_exp = 2
query = f'{job_title} {years_of_exp} jobs site:indeed.com'
links_response = tavily_client.search(query, search_depth="advanced", max_results=5)
job_urls = [r["url"] for r in links_response["results"]]

extract_response = tavily_client.extract(urls=job_urls, format="text", extract_depth="advanced")

final_jds = []
for result in extract_response.get("results", []):
    title = result.get("title", "")
    jd = result.get("description") or result.get("content") or result.get("raw_content")
    if job_title.lower() in title.lower():
        final_jds.append(jd)
print(final_jds)

["Post a Job\n\nHome\nHow to Write a Job Description\nData Scientist\n\nSpecial Offer\nJumpstart your hiring with a $75 credit to sponsor your first job.\nSponsored Jobs are 2.6x times faster to first hire than non-sponsored jobs.\n\nAttract the talent you’re looking for\nGet more visibility in search results\nAppear to more candidates longer\n\nThe $75 Sponsored Job credit offer is only available for new accounts in the US that post a job and expires one year after account creation. Upon expiration of the credit, users are charged based on Sponsored Job budget. Terms, conditions, and quality standards apply Indeed data, based on median United States, Q2 2023\nClaim your $75 credit\nData Scientist Job Description: Top Duties and Qualifications\nEdited by Indeed Employer Content Team\nLast updated March 4, 2025\nA Data Scientist, or Data Science Professional, is responsible for overseeing the collection, storage and interpretation of data for businesses. Their duties include sifting thr

In [5]:
from pydantic import BaseModel, Field

class SkillResponse(BaseModel):
    skills : list[str] = Field(description='list of all skills')
    desc : str = Field(description="Summarized description of all skills")

In [6]:
structured_op = model.with_structured_output(SkillResponse)
prompt = f"read the job description and give me key skills {final_jds}"

msg = structured_op.invoke(prompt)

In [7]:
msg

SkillResponse(skills=['Python', 'R', 'SQL', 'Machine Learning (Supervised, Unsupervised, Deep Learning)', 'Natural Language Processing (NLP)', 'Statistical Analysis', 'Predictive Modeling', 'Data Mining', 'Data Visualization (e.g., Power BI, Tableau)', 'Big Data Technologies (e.g., Hadoop, Spark, Kafka)', 'Cloud Platforms (e.g., Azure, AWS)', 'Data Management (ETL, Data Warehousing, Data Governance)', 'Database Programming', 'Analytical and Problem-Solving Skills', 'Communication and Presentation Skills', 'Data Modeling', 'Feature Engineering', 'Model Deployment (MLOps)', 'Microsoft Excel (Advanced)'], desc='The key skills for a Data Scientist include strong proficiency in programming languages such as Python, R, and SQL, along with extensive experience in various machine learning techniques including supervised, unsupervised, and deep learning. Expertise in statistical analysis, predictive modeling, and data mining is crucial. Data visualization using tools like Power BI or Tableau, a

In [8]:
class PriorityResponsse(BaseModel):
    high_priority : list[str] = Field(description="This contains high_priority skills")
    medium_priority : list[str] = Field(description="This contains medium_priority skills")
    low_priority : list[str] = Field(description="This contains low_priority")

In [9]:
structured_op = model.with_structured_output(PriorityResponsse)
prompt = f"Read all the skills {msg.skills} provided and Job Descriptions {final_jds}, based on that provide the priority level. "

msg = structured_op.invoke(prompt)

In [10]:
msg

PriorityResponsse(high_priority=['Python', 'Machine Learning (Supervised, Unsupervised, Deep Learning)', 'Statistical Analysis', 'Predictive Modeling', 'SQL', 'Analytical and Problem-Solving Skills', 'Communication and Presentation Skills'], medium_priority=['Data Visualization (e.g., Power BI, Tableau)', 'Natural Language Processing (NLP)', 'Data Mining', 'Data Management (ETL, Data Warehousing, Data Governance)', 'Data Modeling', 'Model Deployment (MLOps)', 'Cloud Platforms (e.g., Azure, AWS)', 'R'], low_priority=['Big Data Technologies (e.g., Hadoop, Spark, Kafka)', 'Database Programming', 'Feature Engineering', 'Microsoft Excel (Advanced)'])

In [11]:
msg.high_priority

['Python',
 'Machine Learning (Supervised, Unsupervised, Deep Learning)',
 'Statistical Analysis',
 'Predictive Modeling',
 'SQL',
 'Analytical and Problem-Solving Skills',
 'Communication and Presentation Skills']

In [12]:
msg.medium_priority


['Data Visualization (e.g., Power BI, Tableau)',
 'Natural Language Processing (NLP)',
 'Data Mining',
 'Data Management (ETL, Data Warehousing, Data Governance)',
 'Data Modeling',
 'Model Deployment (MLOps)',
 'Cloud Platforms (e.g., Azure, AWS)',
 'R']

In [13]:
msg.low_priority

['Big Data Technologies (e.g., Hadoop, Spark, Kafka)',
 'Database Programming',
 'Feature Engineering',
 'Microsoft Excel (Advanced)']

In [36]:
class JobAnalyzer(TypedDict):
    job_desc : list[str]
    skills : list[str]
    job_title: str
    yoe: int
    high_priority: list[str] 
    medium_priority : list[str] 
    low_priority : list[str]

In [37]:
from pydantic import BaseModel, Field

class SkillResponse(BaseModel):
    skills : list[str] = Field(description='list of all skills')
    desc : str = Field(description="Summarized description of all skills")

In [38]:
class PriorityResponsse(BaseModel):
    high_priority : list[str] = Field(description="This contains high_priority skills")
    medium_priority : list[str] = Field(description="This contains medium_priority skills")
    low_priority : list[str] = Field(description="This contains low_priority")

In [40]:
def extract_jd(state: JobAnalyzer) -> JobAnalyzer:
    job_title = state["job_title"]
    years_of_exp = state["yoe"]
    query = f'{job_title} {years_of_exp} jobs site:indeed.com'
    links_response = tavily_client.search(query, search_depth="advanced", max_results=5)
    job_urls = [r["url"] for r in links_response["results"]]

    extract_response = tavily_client.extract(urls=job_urls, format="text", extract_depth="advanced")

    final_jds = []
    for result in extract_response.get("results", []):
        title = result.get("title", "")
        jd = result.get("description") or result.get("content") or result.get("raw_content")
        if job_title.lower() in title.lower():
            final_jds.append(jd)
    return {'job_desc': final_jds }

def extract_skills(state: JobAnalyzer) -> JobAnalyzer:
    structured_op = model.with_structured_output(SkillResponse)
    prompt = f"read the job description and give me key skills {state['job_desc']}"

    msg = structured_op.invoke(prompt)
    return {'skills': msg.skills }


def set_priority(state: JobAnalyzer) -> JobAnalyzer:
    structured_op = model.with_structured_output(PriorityResponsse)
    prompt = f"Read all the skills {state['skills']} provided and Job Descriptions {state['job_desc']}, based on that provide the priority level. "

    msg = structured_op.invoke(prompt)
    return {'high_priority': msg.high_priority, 'medium_priority':msg.medium_priority, 'low_priority':msg.low_priority}

In [42]:
graph = StateGraph(JobAnalyzer)

graph.add_node("extract_jd",extract_jd)
graph.add_node("extract_skills",extract_skills)
graph.add_node("set_priority",set_priority)

graph.add_edge(START, "extract_jd")
graph.add_edge("extract_jd","extract_skills")
graph.add_edge("extract_skills","set_priority")
graph.add_edge("set_priority",END)

workflow = graph.compile()


In [43]:
init_state = {
    'job_title' : "Data Scientist",
    'yoe':2
}
workflow.invoke(init_state)

{'job_desc': ["Post a Job\n\nHome\nHow to Write a Job Description\nData Scientist\n\nSpecial Offer\nJumpstart your hiring with a $75 credit to sponsor your first job.\nSponsored Jobs are 2.6x times faster to first hire than non-sponsored jobs.\n\nAttract the talent you’re looking for\nGet more visibility in search results\nAppear to more candidates longer\n\nThe $75 Sponsored Job credit offer is only available for new accounts in the US that post a job and expires one year after account creation. Upon expiration of the credit, users are charged based on Sponsored Job budget. Terms, conditions, and quality standards apply Indeed data, based on median United States, Q2 2023\nClaim your $75 credit\nData Scientist Job Description: Top Duties and Qualifications\nEdited by Indeed Employer Content Team\nLast updated March 4, 2025\nA Data Scientist, or Data Science Professional, is responsible for overseeing the collection, storage and interpretation of data for businesses. Their duties includ